# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(6)
# CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 50

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
  'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
#  'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[2.027 1.996 2.017 1.962 1.912 1.986 2.003 1.945 1.974 1.991]
 [3.003 3.108 2.947 3.025 2.979 2.908 3.059 3.009 3.058 3.03 ]
 [4.032 3.964 3.911 3.998 4.075 4.004 3.982 3.982 3.905 3.928]] [[0.932271 1.009984 1.030711 0.940556 0.954256 0.997804 1.010991 0.957975
  0.981324 1.092919]
 [1.976991 2.116336 1.928191 1.992375 2.010559 1.835536 2.091519 2.086919
  1.998636 2.0971  ]
 [2.902976 2.984704 2.877079 3.079996 3.093375 2.951984 2.967676 2.909676
  2.747975 3.148816]]
Mean and variance endomwnets:
[[2.027 2.073 1.958 2.002 2.017 2.017 2.044 2.007 1.99  1.969]
 [3.033 2.98  3.034 2.898 2.995 3.072 2.986 3.011 3.074 3.019]
 [3.969 4.065 4.036 4.093 3.981 4.041 3.932 3.932 3.975 3.963]] [[1.044271 1.107671 1.000236 1.005996 1.020711 1.058711 1.034064 0.920951
  1.0159   0.948039]
 [2.055911 1.9156   2.020844 2.023596 2.036975 2.066816 1.993804 1.848879
  1.996524 2.068639]
 [2.710039 3.070775 3.164704 3.054351 2.818639 3.017319 2.947376 2.897376
  2.978375

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = { 'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG),
                    'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG),
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG),
#                     'Budget': lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)
                    }
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold


ValueError: too many values to unpack (expected 2)

In [ ]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [ ]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)